<a href="https://colab.research.google.com/github/Mrezakhodashenas/PhD-NN/blob/Mrezakhodashenas-Noise_Investigation/Noisy_intermediate_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reset -f

In [ ]:
!pip install torch

In [ ]:
!pip install wandb --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.6 MB/s eta 0:00:00


In [ ]:
!pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 22.8 MB/s eta 0:00:00


In [ ]:
!pip install snntorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 10.5 MB/s eta 0:00:00


In [ ]:
!pip install torchsummary

In [ ]:
!pip install wandb onnx -Uq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 67.3 MB/s eta 0:00:00


## set seeds for PyTorch and Numpy to ensure reproducibility:

In [ ]:
import torch
import random
import numpy as np

# # Set seeds for Python, Numpy, and Torch for reproducibility
# # Ensure deterministic behavior
# torch.backends.cudnn.deterministic = True
# random.seed(hash("setting random seeds") % 2**32 - 1)
# np.random.seed(hash("improves reproducibility") % 2**32 - 1)
# torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
# torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)


seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Additional steps if you're using GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [ ]:
# imports
import pickle
import matplotlib.animation as animation
from scipy.integrate import simps
import torch
torch.cuda.empty_cache()
import os, sys, time, datetime, json, random
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import utils as utls
from snntorch import utils
from snntorch import surrogate
import numpy as np
import math
from sklearn.metrics import auc
from torchsummary import summary
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import torch.nn as nn
from scipy.stats import entropy
from scipy.special import kl_div
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
# from skopt import gp_minimize
# from bayes_opt import BayesianOptimization
import wandb
from tqdm.auto import tqdm
import matplotlib.ticker as mtick



# import spikeflow as snn

In [ ]:
# fe7dfc869c7793392e4f225d9d64f615e2fd70ec
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
wandb.init(project="Noisy_StdDev0_1_Intermediate_Layers")      # Set the project where this run will be logged

config = dict(
    std_dev = 0.1,
    subset_fraction = 1,     # the percentage of the data used
    epochs=50,

    batch_size=250,
    learning_rate=0.0001,
    input_size = 32,
    kernels=[32, 64, 128],
    threshold_Real = 0.5,
    threshold_Recon = 0.5,

    alpha=0.9,
    # beta_syn=0.0001,
    beta_syn=0.9,
    beta =0.9,
    num_steps=5,
    latent_dim = 32, #dimension of latent layer (how compressed we want the information)
    thresh=1,    #spiking threshold (lower = more spikes are let through)

    lr=0.0001,
    betas=(0.9, 0.999),
    weight_decay=0.001,

    dataset="MNIST",
    architecture="SAE(L_Syn_Syn_L)")

wandb: Currently logged in as: mreza-khodashenas (neuroscience_ai). Use `wandb login --relogin` to force relogin


## set seeds for PyTorch and Numpy to ensure reproducibility:

In [ ]:
start_time = datetime.datetime.now()

# /////////////////////# Building the Autoencoder
#-------------------DataLoaders.  using the MNIST dataset

# dataloader arguments
batch_size = 250
data_path='/data/mnist'

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


# /////////////////////////////////# Define a transform
input_size = 32 # resizing the original MNIST from 28 to 32

transform = transforms.Compose([
            transforms.Resize((input_size, input_size)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

#------------------------------------------- Load MNIST
# Training data
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Testing data
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)



100%|██████████| 9912422/9912422 [00:00<00:00, 210795972.48it/s]

Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 12975117.16it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 76205650.46it/s]

Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 10624946.33it/s]


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [ ]:
# creating directories where we can save the original and reconstructed images for training and testing:
# create training/ and testing/ folders in the chosen path
if not os.path.isdir('figures/training'):
    os.makedirs('figures/training')
if not os.path.isdir('figures/binarytraining'):
    os.makedirs('figures/binarytraining')

if not os.path.isdir('figures/testing'):
    os.makedirs('figures/testing')
if not os.path.isdir('figures/binarytesting'):
    os.makedirs('figures/binarytesting')


if not os.path.isdir('Saved_Trained_Checkpoints/'):
    os.makedirs('Saved_Trained_Checkpoints/')

if not os.path.isdir('Output_Spikes/'):
    os.makedirs('Output_Spikes/')

if not os.path.isdir('Enc_syn_Spikes/'):
    os.makedirs('Enc_syn_Spikes/')


if not os.path.isdir('Intermediate_Lyrs/'):
    os.makedirs('Intermediate_Lyrs/')



In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using {torch.cuda.get_device_name()} ({device})")
else:
    device = torch.device("cpu")
    print("Using CPU")


Using Tesla T4 (cuda)


In [ ]:
class SAE(nn.Module):
    def __init__(self):
        super().__init__()

        # Encoder
        self.encoder = nn.Sequential(
                            nn.Conv2d(1, 32, 3,padding = 1,stride=2), # Conv Layer 1
                            nn.BatchNorm2d(32),
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.Conv2d(32, 64, 3,padding = 1,stride=2), # Conv Layer 2
                            nn.BatchNorm2d(64),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True, threshold=thresh), #SNN TORCH LIF NEURON
                            nn.Conv2d(64, 128, 3,padding = 1,stride=2), # Conv Layer 3
                            nn.BatchNorm2d(128),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh), #SNN TORCH LIF NEURON
                            nn.Flatten(start_dim = 1, end_dim = 3), #Flatten convolutional output
                            nn.Linear(128*4*4, latent_dim), # Fully connected linear layer
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh)
                            )


        self.latent_dim = latent_dim #dimensions of the encoded z-space data
        self.linearNet= nn.Sequential(
                                      nn.Linear(latent_dim,128*4*4),
                                      snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True,threshold=thresh))

        # Decoder:
        self.decoder = nn.Sequential(
                            nn.Unflatten(1,(128,4,4)), #Unflatten data from 1 dim to tensor of 128 x 4 x 4
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.ConvTranspose2d(128, 64, 3,padding = 1,stride=(2,2),output_padding=1),
                            nn.BatchNorm2d(64),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.ConvTranspose2d(64, 32, 3,padding = 1,stride=(2,2),output_padding=1),
                            nn.BatchNorm2d(32),
                            snn.Synaptic(alpha=alpha, beta=beta_syn, spike_grad=spike_grad, init_hidden=True,threshold=thresh),
                            nn.ConvTranspose2d(32, 1, 3,padding = 1,stride=(2,2),output_padding=1),
                            snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True,output=True,threshold=20000) #so membrane can be trained
                            )
    def forward(self, x):
        utils.reset(self.encoder) #need to reset the hidden states of LIF
        utils.reset(self.decoder)
        utils.reset(self.linearNet)

    #-----------------------------encode
        spk_mem=[];
        spk_rec=[];
        spk_rec_syn=[];
        encoder_mem=[];
        spk_rec_dec=[];
        spk_mem_dec=[];
        enc5_rec = [];


     #------------------------------ intermediate layers

        # for step in range(num_steps):
        #     enc5 = self.encoder[5](x)             #  enc5 shape: torch.Size([250, 1, 32, 32])
        #     enc5_rec.append(enc5)
        # enc5_rec = torch.stack(enc5_rec, dim=2)            #   enc5_rec size: torch.Size([250, 1, 5, 32, 32])
        # Enc_syn_1 = enc5_rec[:, :, -1]                      # #   torch.Size([250, 1, 32, 32])

     #------------------------------ encode
        for step in range(num_steps):
            spk_x, mem_x = self.encoder(x)              # spk_x size: ([250, 32])  ,   mem_x size: ([250, 32])  , x.shape : torch.Size([250, 1, 32, 32])
            spk_rec.append(spk_x)
            spk_mem.append(mem_x)

        spk_rec=torch.stack(spk_rec,dim=2) # stack spikes in second tensor dimension # ----------------spk_rec in torch.stack(spk_rec,dim=2):  torch.Size([250, 32, 5])
        spk_mem=torch.stack(spk_mem,dim=2) # stack membranes in second tensor dimension # ----------------spk_mem in torch.stack(spk_mem,dim=2):  torch.Size([250, 32, 5])
        out_en = spk_rec[...,step]

        # print("out_en= spk_rec[...,step]:-----------" , spk_rec[...,step].size()) # spk_rec[...,step]:----------- torch.Size([250, 32])       input of the latent and then decoder

     #------------------------------decode
        spk_mem2=[];
        spk_rec2=[];
        decoded_x=[];
        spk_x_dec=[];
        mem_x_dec=[];
        for step in range(num_steps): #for t in time                           #        from decoder: ([250, 1, 32, 32])
            x_recon, x_mem_recon = self.decode(spk_rec[...,step])
            spk_rec2.append(x_recon)
            spk_mem2.append(x_mem_recon)

        spk_rec2=torch.stack(spk_rec2,dim=4)
        spk_mem2=torch.stack(spk_mem2,dim=4)

        out = spk_mem2[:,:,:,:,-1]

        self.out_en = out_en
        self.out = out

        return out, out_en
        # return out, Enc_syn_1

    def encode(self,x):
      spk_latent_x, mem_latent_x = self.encoder(x)
      return spk_latent_x, mem_latent_x


    def decode(self,x):
        spk_x, mem_x = self.linearNet(x) #convert latent dimension back to total size of features in encoder final layer
        spk_x2, mem_x2 = self.decoder(spk_x)
        return spk_x2, mem_x2



In [ ]:
import numpy as np

def calculate_pixel_accuracy(real_img, x_recon):
    # Flatten the tensors and convert them to NumPy arrays
    real_img_np = real_img.flatten().cpu().numpy()
    x_recon_np = x_recon.flatten().detach().cpu().numpy()

    # Calculate pixel-wise similarity
    pixel_accuracy = 1 - np.mean(np.abs(x_recon_np - real_img_np))

    return pixel_accuracy


In [ ]:

# Training and Testing
# using MSE loss to compare the reconstructed image (x_recon) with the original image (real_img)
from torchvision.utils import save_image

spike_recordings = []
train_ber_rec = []
test_ber_rec = []
threshold_Real = 0.5
threshold_Recon = 0.5

def train(network, trainloader, opti, epoch):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(network, log="all", log_freq=10)
    network=network.train()
    train_loss_hist=torch.zeros((1), dtype=dtype, device=device)
    train_avg_loss_rec=[]
    train_loss_rec = []
    train_accuracy = []

    for batch_idx, (real_img, labels) in enumerate(trainloader):
        opti.zero_grad()
        real_img = real_img.to(device)
        labels = labels.to(device)

      # -------------Add Gaussian noise to the input images
        real_img = real_img+(torch.randn_like(real_img) * std_dev)


        out, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
        x_recon, out = network(real_img)  # Pass data into network and return reconstructed image and spk_rec.  #        x_recon size torch.Size([250, 1, 32, 32]) ,  #        out size torch.Size([250, 32])
        #Calculate loss
        loss_val = torch.zeros((1), dtype=dtype, device=device)
        for step in range(num_steps):
          loss_val += F.mse_loss(x_recon, real_img)                  #.view(1, -1)
        train_loss_hist += (loss_val.item())/num_steps
        avg_loss=train_loss_hist.mean()

        # # ---------------------------- Calculate Bit Error Rate (BER)
        real_img_binary = (real_img > threshold_Real).float()
        x_recon_binary = (x_recon > threshold_Recon).float()
        bit_errors = torch.sum(torch.abs(real_img_binary - x_recon_binary))
        total_pixels = real_img_binary.numel()  # Total number of pixels in the images
        bit_error_rate = bit_errors.item() / total_pixels
        train_ber_rec.append(bit_error_rate)  # Append BER to the list

        # # --------------------------------- Calculate Accuracy------------
        # # # accuracy = accuracy_score(real_img.flatten().cpu().numpy(), x_recon.flatten().detach().cpu().numpy())
        train_accuracy_pix = calculate_pixel_accuracy(real_img, x_recon)
        train_accuracy.append(train_accuracy_pix.item())

        # ---------------
        loss_val.backward()
        opti.step()
        train_loss_rec.append(loss_val.item())
        # ----------------



        # Save binary images
        Error_bin = (torch.abs(x_recon_binary - real_img_binary))

        if batch_idx == len(trainloader)-1:
          if epoch in [0, 49]:
            utls.save_image(real_img_binary, f'figures/binarytraining/Train_ep{epoch}_inputs_binary.png')
            utls.save_image(x_recon_binary, f'figures/binarytraining/Train_ep{epoch}_recon_binary.png')
            utls.save_image(Error_bin, f'figures/binarytraining/Train_ep{epoch}_Error_bin.png')

        # Logging for wandb
        if (batch_idx + 1) % 10 == 0:
          wandb.log({"Epoch": epoch, "Train_Batch": batch_idx, "Train_Loss": loss_val.item(), "Train_BER": bit_error_rate, "Train_Acc": train_accuracy_pix* 100})


        print(f'Train[{epoch}/{max_epoch}][{batch_idx}/{len(trainloader)}] Train_Loss: {loss_val.item()}, ' f'Train_BER : {bit_error_rate}, Train_Acc: {train_accuracy_pix*100:.4f}%')


        #Save reconstructed images every at the end of the epoch
        if batch_idx == len(trainloader)-1:
          if epoch in [0, 49]:
            utls.save_image((real_img+1)/2, f'figures/training/Train_epoch{epoch}_finalbatch_inputs.png')
            utls.save_image((x_recon+1)/2, f'figures/training/Train_epoch{epoch}_finalbatch_recon.png')
            train_auc = auc(np.arange(len(train_loss_rec)), train_loss_rec)

    epoch_loss = sum(train_loss_rec) / len(train_loss_rec)
    epoch_accuracy = sum(train_accuracy) / len(train_accuracy)
    print(f'Epoch [{epoch}/{max_epoch}] Train_Avg_loss(epoch): {epoch_loss:.5f}, Train_AvgAcc(epoch): {epoch_accuracy*100:.4f}%')
    print('-------------------------------------')
    wandb.log({"Epoch": epoch, "Train_AvgLoss(Epoch)": epoch_loss, "Train_AvgAcc(Epoch)": epoch_accuracy * 100})

    # return loss_val, train_loss_rec, train_auc , out, out_en  #, spk_rec_batches#, train_avg_loss_rec, #avg_loss #, train_loss_hist
    return loss_val, train_loss_rec, epoch_loss, train_accuracy, epoch_accuracy, out, out_en      # train_accuracy, epoch_accuracy, noisy_out_en -----------------added

    # return loss_val, train_loss_rec, train_auc   #


# For Testing, not doing backpropagate, therefore no gradients are required and we use torch.no_grad():
#Testing Loop
def test(network, testloader, opti, epoch):
    network=network.eval()
    test_loss_hist=[]
    test_avg_loss_rec=[]
    test_avg_loss_hist = []
    test_accuracy = []  # Define test_accuracy list

    spk_rec_test = []
    spk_rec_test = []
    with torch.no_grad(): #no gradient this time
        for batch_idx, (real_img, labels) in enumerate(testloader):
            real_img = real_img.to(device)#
            labels = labels.to(device)

            out, out_en = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
            x_recon , out = network(real_img)  # Pass data into network and return reconstructed image and spk_rec
            # average Loss:
            loss_val = torch.zeros((1), dtype=dtype, device=device)
            for step in range(num_steps):
              loss_val += F.mse_loss(x_recon, real_img)

            avg_loss=loss_val/num_steps
            test_loss_hist.append(loss_val.item())

            real_img_binary = (real_img > threshold_Real).float()
            x_recon_binary = (x_recon > threshold_Recon).float()
            bit_errors = torch.sum(torch.abs(real_img_binary - x_recon_binary))
            total_pixels = real_img_binary.numel()  # Total number of pixels in the images
            bit_error_rate = bit_errors.item() / total_pixels
            test_ber_rec.append(bit_error_rate)  # Append BER to the list


            # #----------------------------------------------------- Calculate accuracy
            # # # accuracy = accuracy_score(real_img.flatten().cpu().numpy(), x_recon.flatten().detach().cpu().numpy())
            test_accuracy_pix = calculate_pixel_accuracy(real_img, x_recon)
            test_accuracy.append(test_accuracy_pix.item())  # Record accuracy

            # ----------------------------------------------------------------
            # Save binary images
            Error_bin = (torch.abs(x_recon_binary - real_img_binary))

            if batch_idx == len(testloader)-1:
              if epoch in [0, 49]:
                save_image(real_img_binary, f'figures/binarytesting/Test_ep{epoch}_inputs_binary.png')
                save_image(x_recon_binary, f'figures/binarytesting/Test_ep{epoch}_recon_binary.png')
                save_image(Error_bin, f'figures/binarytesting/Test_ep{epoch}_Error_bin.png')


        # Logging for wandb
            if (batch_idx + 1) % 10 == 0:
              wandb.log({"Epoch": epoch, "Test_Batch": batch_idx, "Test_Loss": loss_val.item(), "Test_BER": bit_error_rate, "Test_Acc": test_accuracy_pix* 100})

            # -------------------------------------------------------------------------------------------------

            print(f'Test[{epoch}/{max_epoch}][{batch_idx}/{len(testloader)}]  Loss: {loss_val.item()}, '  f'BER (test): {bit_error_rate},  Accuracy(test): {test_accuracy_pix*100:.4f}%')

            test_loss_rec.append(loss_val.item())

            if batch_idx == len(testloader)-1:
              if epoch in [0, 49]:
                utls.save_image((real_img+1)/2, f'figures/testing/Test_epoch{epoch}_finalbatch_inputs.png')
                utls.save_image((real_img+(torch.randn_like(real_img) * std_dev)+1)/2, f'figures/testing/Test_(real_img+noise)_epoch{epoch}_finalbatch_inputs.png')
                utls.save_image((x_recon+1)/2, f'figures/testing/Test_epoch{epoch}_finalbatch_recons.png')
                test_auc = auc(np.arange(len(test_loss_rec)), test_loss_rec)

    epoch_loss_test = sum(test_loss_hist) / len(test_loss_hist)
    epoch_accuracy_test = sum(test_accuracy) / len(test_accuracy)
    print(f'Test_Epoch [{epoch}/{max_epoch}] Test_Avg_loss(epoch): {epoch_loss_test:.5f}, Average Accuracy_test (epoch): {epoch_accuracy_test*100:.4f}%')

    wandb.log({"Test_Epoch": epoch, "Test_AvgLoss(Epoch)": epoch_loss_test, "Test_AvgAcc(Epoch)": epoch_accuracy_test * 100})

    # return loss_val, test_loss_rec, test_auc, out, out_en                     # -------------------------------------------------ADDED
    return loss_val, test_loss_rec, test_accuracy, epoch_loss_test, epoch_accuracy_test, out, out_en     # ------------------------------------- noisy_out_en  , test_accuracy ------------------ADDED

    # return loss_val, test_loss_rec, test_auc  #

for batch_spikes in spike_recordings:
    print(batch_spikes)

In [ ]:
input_size = 32 #resize of mnist data (optional)

#setup GPU
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# neuron and simulation parameters
spike_grad = surrogate.atan(alpha=2.0)  # alternate surrogate gradient fast_sigmoid(slope=25)

train_loss_rec = []
test_loss_rec = []
train_loss_record = []
test_loss_record = []
train_avg_loss_rec=[]
test_avg_loss_rec=[]

  # Synaptic current and membrane potential decay exponentially with rates of alpha and beta
alpha=0.9
# beta_syn=0.0001
beta_syn=0.9
beta =0.9
std_dev=0.1


num_steps=5
latent_dim = 32 #dimension of latent layer (how compressed we want the information)
thresh=1    #spiking threshold (lower = more spikes are let through)
epochs=50
# epochs=5
max_epoch=epochs

  #Define Network and optimizer
net=SAE()
net = net.to(device)
optimizer = torch.optim.AdamW(net.parameters(),
                            lr=0.0001,
                            betas=(0.9, 0.999),
                            weight_decay=0.001)



activation = {}
# def get_activation(name):
#     def hook(model, input, output):
#         activation[name] = output.detach()
#     return hook


def get_activation(name):
    def hook(model, input, output):
        if isinstance(output, tuple):
            activation[name] = [out.detach() for out in output]
        else:
            activation[name] = output.detach()
    return hook


# net.encoder[5].register_forward_hook(get_activation('encoder[5]'))


## for saving after each epoch:

In [ ]:
checkpoint_path = "Saved_Trained_Checkpoints/"
Output_Spikes = "Output_Spikes/"
Enc_syn_Spikes = "Enc_syn_Spikes/"
Intermediate_Lyrs = "Intermediate_Lyrs/"
epoch_activations_list = [] # Create a list to store activations for each epoch
epoch_activations = {}

# ///////////////////////////////////

# Define hook_layers and hook_names
hook_layers = [net.encoder[2], net.encoder[5], net.encoder[8], net.encoder[11], net.decoder[1], net.decoder[4], net.decoder[7], net.decoder[9]]
hook_names = ["Enc_Lk1", "Enc_syn1", "Enc_syn2", "Enc_Lk2", "Dec_Lk1", "Dec_syn1", "Dec_syn2", "Dec_Lk2"]

# Create an empty dictionary to store activations
epoch_activations = {}

# Register hooks for capturing activations
hooks = []
for i, layer in enumerate(hook_layers):
    hook_fn = get_activation(hook_names[i])
    hooks.append(layer.register_forward_hook(hook_fn))


# -----------------------------------------------------------
import os

# Run training and testing
for e in range(epochs):
    train_loss = train(net, train_loader, optimizer, e)
    train_avg_loss_rec.append(sum(train_loss_rec) / len(train_loader))

    test_loss = test(net, test_loader, optimizer, e)
    test_avg_loss_rec.append(sum(test_loss_rec) / (len(test_loader)))

     #----------Save the model every 10 epochs
    if (e + 1) % 10 == 0:
       model_path = checkpoint_path + f"Benchmark_StdDev0_1_model_epoch_{e + 1}.pt"
       torch.save(net.state_dict(), model_path)

    # #---------------------------------------------------------- Access the out_en tensor
    out_en = net.out_en
    out_en_numpy = out_en.cpu().detach().numpy()
    # Save with a different name for each epoch
    out_en_filename = Output_Spikes + f"Benchmark_StdDev0_1_out_en_epoch_{e + 1}.npy"
    np.save(out_en_filename, out_en_numpy)

    # #-----------------------------------------------------------Access the out tensor
    out = net.out
    out_numpy = out.cpu().detach().numpy()
    # Save with a different name for each epoch
    out_filename = Output_Spikes + f"Benchmark_StdDev0_1_out_epoch_{e + 1}.npy"
    np.save(out_filename, out_numpy)
    # -------------------------------------------------------------------- Intermediate Layers
# Check if the current epoch is a multiple of 10
    if (e + 1) % 10 == 0:
        # Save the epoch_activations dictionary to a file
        activations_path = Intermediate_Lyrs +  f"Benchmark_StdDev0_1_activations_epoch_{e + 1}.pkl"
        with open(activations_path, 'wb') as file:
            pickle.dump(epoch_activations, file)
    # Capture activations for the current epoch
    epoch_activations[e] = {}
    for i, name in enumerate(hook_names):
        epoch_activations[e][name] = activation.get(name, None)  # Use get to avoid KeyError




# beta_syn=0.9


Streaming output truncated to the last 5000 lines.
Train[32/50][94/240] Train_Loss: 0.11549282819032669, Train_BER : 0.04335546875, Train_Acc: 88.8979%
Train[32/50][95/240] Train_Loss: 0.11407387256622314, Train_BER : 0.0425390625, Train_Acc: 88.9425%
Train[32/50][96/240] Train_Loss: 0.11246125400066376, Train_BER : 0.041671875, Train_Acc: 89.0084%
Train[32/50][97/240] Train_Loss: 0.11525438725948334, Train_BER : 0.0433203125, Train_Acc: 88.9177%
Train[32/50][98/240] Train_Loss: 0.11308345198631287, Train_BER : 0.04235546875, Train_Acc: 88.9888%
Train[32/50][99/240] Train_Loss: 0.11025956273078918, Train_BER : 0.04076171875, Train_Acc: 89.0938%
Train[32/50][100/240] Train_Loss: 0.11353228241205215, Train_BER : 0.04107421875, Train_Acc: 89.0073%
Train[32/50][101/240] Train_Loss: 0.11730984598398209, Train_BER : 0.04451171875, Train_Acc: 88.8071%
Train[32/50][102/240] Train_Loss: 0.11696778237819672, Train_BER : 0.043359375, Train_Acc: 88.8391%
Train[32/50][103/240] Train_Loss: 0.1182618

In [ ]:
# ////////////////////////////////////// just for printing elapsed time
def format_time(seconds):
    if seconds < 300:
        s = float(seconds)
        return "%.1f seconds" % (s,)
    elif seconds < 3600:
        m = seconds / 60.0
        return "%.2f minutes" % (m,)
    else:
        h = seconds / 3600.0
        return "%.2f hours" % (h,)

dt = datetime.datetime.now() - start_time
t = format_time(dt.total_seconds())

end_time = datetime.datetime.now()
dt = datetime.datetime.now() - start_time
seconds = dt.total_seconds()
t = format_time(seconds)
print("time: %s " % (t))
# /////////////////////////////////////////////////

time: 1.61 hours 


In [ ]:


#---------------------------------------------- Benchmark ----------------------------------------
print('------------------------------------Benchmark------------------------------------------')
print('-----////////////random.seed(hash("setting random seeds") % 2**32 - 1)/////////\n')
print('Train[49/50][239/240] Loss: 0.10289572924375534, BER : 0.03852734375')
print('Epoch [49/50] Train_Avg_loss(epoch): 0.23784')
print('----------------test: ---------------------')
print('Test[49/50][39/40]  Loss: 0.06783822178840637, BER (test): 0.0381953125')
print('Test_Epoch [49/50] Test_Avg_loss(epoch): 0.06835')

print('\n--------/////////////////////seed = 42//////////////////-------\n')
print('Train[49/50][239/240] Train_Loss: 0.1043865829706192, Train_BER : 0.040484375, Train_Acc: 89.4216%')
print('Epoch [49/50] Train_Avg_loss(epoch): 0.10296, Train_AvgAcc(epoch): 89.4573%')
print('----------------test: ---------------------')
print('Test[49/50][39/40]  Loss: 0.08995474129915237, BER (test): 0.04503515625,  Accuracy(test): 90.4160%')
print('Test_Epoch [49/50] Test_Avg_loss(epoch): 0.08853, Average Accuracy_test (epoch): 90.4677%\n')

print('\n--------/////////////////////seed = 42//////////////////-------')
print('Train[49/50][239/240] Train_Loss: 0.10537073016166687, Train_BER : 0.039671875, Train_Acc: 89.3635%')
print('Epoch [49/50] Train_Avg_loss(epoch): 0.10403, Train_AvgAcc(epoch): 89.3979%')
print('----------------test: ---------------------')
print('Test[49/50][39/40]  Loss: 0.09431052207946777, BER (test): 0.04759375,  Accuracy(test): 89.9162%')
print('Test_Epoch [49/50] Test_Avg_loss(epoch): 0.09267, Average Accuracy_test (epoch): 89.9526%%\n')




print('--------////////////////////////seed = NO///////////////-------\n')

#---------------------------------------------- Overfiring ----------------------------------------
print('\n')
print('------------------------------------Overfiring------------------------------------------')
print('\n')
print('-----////////////random.seed(hash("setting random seeds") % 2**32 - 1)/////////\n')
print('Train[49/50][239/240] Loss: 0.112606942653656, BER : 0.041515625')
print('Epoch [49/50] Train_Avg_loss(epoch): 0.20606')
print('----------------test: ---------------------')
print('Test[49/50][39/40]  Loss: 0.09274159371852875, BER (test): 0.04833984375')
print('Test_Epoch [49/50] Test_Avg_loss(epoch): 0.09305')
print('\n')

print('--------////////////////////////seed = 42///////////////-------\n')
print('Train[49/50][239/240] Train_Loss: 0.13240158557891846, Train_BER : 0.0505390625, Train_Acc: 88.2287%')
print('Epoch [49/50] Train_Avg_loss(epoch): 0.12977, Train_AvgAcc(epoch): 88.3128%')
print('----------------test: ---------------------')
print('Test[49/50][39/40]  Loss: 0.09381721913814545, BER (test): 0.04915234375,  Accuracy(test): 91.3096%')
print('Test_Epoch [49/50] Test_Avg_loss(epoch): 0.09290, Average Accuracy_test (epoch): 91.3520%\n')

print('--------////////////////////////seed = NO///////////////-------\n')
print('Train[49/50][239/240] Train_Loss: 0.1134030818939209, Train_BER : 0.0409375, Train_Acc: 88.9049%')
print('Epoch [49/50] Train_Avg_loss(epoch): 0.11575, Train_AvgAcc(epoch): 88.8131%')
print('----------------test: ---------------------')
print('Test[49/50][39/40]  Loss: 0.09596434235572815, BER (test): 0.0480078125,  Accuracy(test): 90.9138%')
print('Test_Epoch [49/50] Test_Avg_loss(epoch): 0.09912, Average Accuracy_test (epoch): 90.7719%')

------------------------------------Benchmark------------------------------------------
-----////////////random.seed(hash("setting random seeds") % 2**32 - 1)/////////

Train[49/50][239/240] Loss: 0.10289572924375534, BER : 0.03852734375
Epoch [49/50] Train_Avg_loss(epoch): 0.23784
----------------test: ---------------------
Test[49/50][39/40]  Loss: 0.06783822178840637, BER (test): 0.0381953125
Test_Epoch [49/50] Test_Avg_loss(epoch): 0.06835

--------/////////////////////seed = 42//////////////////-------

Train[49/50][239/240] Train_Loss: 0.1043865829706192, Train_BER : 0.040484375, Train_Acc: 89.4216%
Epoch [49/50] Train_Avg_loss(epoch): 0.10296, Train_AvgAcc(epoch): 89.4573%
----------------test: ---------------------
Test[49/50][39/40]  Loss: 0.08995474129915237, BER (test): 0.04503515625,  Accuracy(test): 90.4160%
Test_Epoch [49/50] Test_Avg_loss(epoch): 0.08853, Average Accuracy_test (epoch): 90.4677%


--------/////////////////////seed = 42//////////////////-------
Train[49/50

In [ ]:

# ------------------------------------ Load from Drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import shutil

# Define paths
source_path = "/content/Intermediate_Lyrs/Benchmark_StdDev0_1_activations_epoch_50.pkl"  # Provide the path to your big file in Colab
destination_path = "/content/drive/MyDrive/Benchmark_StdDev0_1_activations_epoch_50.pkl"  # Destination path in your Google Drive

# Move the file
shutil.move(source_path, destination_path)

'/content/drive/MyDrive/Benchmark_StdDev0_1_activations_epoch_50.pkl'